## Compute_overlap_array is too slow when comparing two annotation volumes

In fact, the time is almost quadratic in relation to the array size and linearly in relation to number of annotations. We can get close to linear time given two annotation volumes by:
  1. Create a tight bounding box around a prediction or a ground truth
  2. Calculate the overlaps only on the bounding box volume.

In [1]:
import numpy as np
from skimage.measure import label

import sys
sys.path.append('../../bstadt/Quality/')

from Quality import compute_overlap_array

In [26]:
import numpy as np
from skimage.measure import label

def bounding_box(img):
    """
    Returns the z, y, x vectors that create a bounding box
    of a mask.
    """
    z = np.any(img, axis=(1, 2))
    y = np.any(img, axis=(0, 2))
    x = np.any(img, axis=(0, 1))

    zmin, zmax = np.where(z)[0][[0, -1]]
    ymin, ymax = np.where(y)[0][[0, -1]]
    xmin, xmax = np.where(x)[0][[0, -1]]

    return (zmin, zmax + 1), (ymin, ymax + 1), (xmin, xmax + 1)


def get_uniques(ar):
    """
    Returns an ordered numpy array of unique integers in an array.
    This runs about four times faster than numpy.unique().

    Parameters
    ----------
    ar : array_like
        Input array. This will be flattened.

    Returns
    -------
    uniques : ndarray
        The sorted unique values.
    """
    bins = np.zeros(np.max(ar) + 1, dtype=int)
    bins[ar.ravel()] = 1
    uniques = np.nonzero(bins)[0]

    return uniques


def get_unique_overlap2(foreground, background, i):
    '''
    Calculates the number of unique background labels in the foreground at i
    Does not count background label of 0
    '''
    z, y, x = bounding_box(foreground == i)
    
    foreground = foreground[z[0]:z[1], y[0]:y[1], x[0]:x[1]]
    background = background[z[0]:z[1], y[0]:y[1], x[0]:x[1]]
    
    overlaps = np.multiply((foreground == i), background)
    uniques = get_uniques(overlaps)

    num_unique = len(uniques)

    #0 is background label
    #should not count as a detection if
    #the prediction overlaps with the background
    if 0 in uniques:
        num_unique -= 1

    return num_unique


def compute_overlap_array2(predictions, gt, compare_annotations=False):
    if not compare_annotations:
        predictions = label(predictions)
        prediction_uniques = get_uniques(predictions)[1:]
    elif compare_annotations:
        prediction_uniques = get_uniques(predictions)

    gt_uniques = get_uniques(gt)[1:]

    #first, look at how many unique predictions
    #overlap with a single gt synapse
    predictionPerGt = [get_unique_overlap2(gt, predictions, i)
                       for i in gt_uniques]

    #next, look at how many unique synapses overlap
    #with a single synapse prediction
    gtPerPrediction = [get_unique_overlap2(predictions, gt, i)
                       for i in prediction_uniques]

    return {'predictionPerGt': predictionPerGt,
            'gtPerPrediction': gtPerPrediction}


In [5]:
file = np.load('../data/collman15v2.npz')
collman_annotation = file['annotation']

file = np.load('../../dmannan/Annotations/annotation_drishti.npz')
drishti_annotation = file['annotation_drishti']

First make sure the outputs are the same

In [27]:
compute_overlap_array(drishti_annotation[0:10, 1000:2000, 1000:2000], \
                           collman_annotation[0:10, 1000:2000, 1000:2000])\
== compute_overlap_array2(drishti_annotation[0:10, 1000:2000, 1000:2000], \
                           collman_annotation[0:10, 1000:2000, 1000:2000])

True

For array size (10, 1000, 1000), it takes:
  1. without bounding box: 1.17s
  2. with bounding box: 409ms

In [15]:
%timeit -n1 compute_overlap_array(drishti_annotation[0:10, 1000:2000, 1000:2000], \
                           collman_annotation[0:10, 1000:2000, 1000:2000])

1.17 s ± 34.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit -n1 compute_overlap_array2(drishti_annotation[0:10, 1000:2000, 1000:2000], \
                           collman_annotation[0:10, 1000:2000, 1000:2000])

409 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


For array size (10, 2000, 2000), which is 4 times larger than (10, 1000, 1000), it takes:
  1. without bounding box: 15.2s
  2. with bounding box: 3.15s

In [17]:
%timeit -n1 compute_overlap_array(drishti_annotation[0:10, 1000:3000, 1000:3000], \
                           collman_annotation[0:10, 1000:3000, 1000:3000])

15.2 s ± 663 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit -n1 compute_overlap_array2(drishti_annotation[0:10, 1000:3000, 1000:3000], \
                           collman_annotation[0:10, 1000:3000, 1000:3000])

3.15 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


For array size (20, 2000, 2000), which is 8 times larger than (10, 1000, 1000), it takes:
  1. without bounding box: 63s
  2. with bounding box: 11.6s

In [19]:
%timeit -n1 compute_overlap_array(drishti_annotation[0:20, 1000:3000, 1000:3000], \
                           collman_annotation[0:20, 1000:3000, 1000:3000])

1min 3s ± 6.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit -n1 compute_overlap_array2(drishti_annotation[0:20, 1000:3000, 1000:3000], \
                           collman_annotation[0:20, 1000:3000, 1000:3000])

11.6 s ± 956 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Not really going to compare the entire volume since it takes about two hours to run.